In [12]:
#!pip3.10 install krippendorff
#!pip3.10 install statsmodels
#!pip3.10 install crowd-kit

In [7]:
import json
import copy
import pandas as pd
import re
from statsmodels.stats.inter_rater import fleiss_kappa, aggregate_raters
import krippendorff as kd
import numpy as np

In [8]:
# from google.colab import drive

# drive.mount('/content/gdrive')

In [13]:
df = pd.read_csv("data/assignments_from_pool_42774131__05-02-2024.tsv", sep="\t")
df = df[df['ASSIGNMENT:status']=='APPROVED']

In [14]:
from crowdkit.metrics.data._classification import alpha_krippendorff

In [16]:
tasks = {}
tasks_labels = []
for index, row in df.iterrows():
    tasks_labels.append({'task': row['ASSIGNMENT:task_id'], 'worker': row['ASSIGNMENT:worker_id'], 'label': int(row['OUTPUT:R-4'])})
    if row["ASSIGNMENT:task_id"] not in tasks:
        tasks[row["ASSIGNMENT:task_id"]] = {}
        tasks[row["ASSIGNMENT:task_id"]]["text"] = row["INPUT:input"]
        tasks[row["ASSIGNMENT:task_id"]]["workers"] = {row["ASSIGNMENT:worker_id"]:row["OUTPUT:R-4"]}
    else:
        tasks[row["ASSIGNMENT:task_id"]]["workers"][row["ASSIGNMENT:worker_id"]] = row["OUTPUT:R-4"]

In [27]:
from collections import Counter

toloker2count = dict(Counter(pd.DataFrame.from_dict(tasks_labels).worker.values.tolist()).most_common())

In [28]:
import json
with open("data/output_annotation.json", 'w') as w:
    json.dump(tasks, w)

In [18]:
pd.DataFrame.from_records(tasks_labels)

,task,worker,label
0,00028cae73--65bfa3f40cadac2361249abc,d7c8ba18b024aadad3c03dc12bceface,2
1,00028cae73--65bfa3a1a15018044b04914f,d7c8ba18b024aadad3c03dc12bceface,2
2,00028cae73--65bfa3f80cadac236124a05f,d7c8ba18b024aadad3c03dc12bceface,2
3,00028cae73--65bfa3f30cadac23612498c8,d7c8ba18b024aadad3c03dc12bceface,2
4,00028cae73--65bfa3f50cadac2361249b93,d7c8ba18b024aadad3c03dc12bceface,2
...,...,...,...
6404,00028cae73--65bfa3f80cadac236124a041,8dc6cecab5a1e798269b109e83f64621,2
6405,00028cae73--65bfa3f30cadac236124994e,8dc6cecab5a1e798269b109e83f64621,2
6406,00028cae73--65bfa3f70cadac2361249f6e,8dc6cecab5a1e798269b109e83f64621,2
6407,00028cae73--65bfa3f70cadac2361249e1f,8dc6cecab5a1e798269b109e83f64621,2


In [19]:
alpha_krippendorff(pd.DataFrame.from_records(tasks_labels))

0.2686509960442679

In [29]:
from collections import Counter

js = []
for i,j in Counter(pd.DataFrame.from_dict(tasks_labels).task.values.tolist()).most_common():
    js.append(j)

Counter(js)

Counter({3: 1709,
         66: 4,
         72: 2,
         63: 2,
         62: 2,
         59: 2,
         76: 1,
         71: 1,
         70: 1,
         68: 1,
         61: 1,
         60: 1,
         51: 1,
         49: 1})

In [21]:
from nltk.tokenize import NLTKWordTokenizer

In [30]:
tags = {1.0: 1, 2.0: 2, 3.0: 3, 4.0: 4, 0.0: 0}

In [31]:
import pandas as pd

# Placeholder for the new rows
rows = []

for n_sent, task in enumerate(tasks):
    text = tasks[task]['text']
    workers = tasks[task]['workers']
    row = {'text': text, 'n_sent': n_sent}
    
    for worker, label in workers.items():
        row[worker] = label
    
    rows.append(row)

# Create the DataFrame at once
df_raters = pd.DataFrame(rows)

# (Optional) Ensure the 'n_sent' column is the correct type
df_raters = df_raters.astype({'n_sent': 'int32'})

In [32]:
df_raters

,text,n_sent,d7c8ba18b024aadad3c03dc12bceface,31e4bb0217a21ef6451faaee8847d7c1,b0e385e880cb3f5033308f1a9d63e5c4,9a8928d0d44faa52cf9e7118122d68a3,646ef569491540d55e1f3140841a1a15,2273fc122c3fd60f9e096cf0afaeabb9,5f1770836609d188639b9b73f25500d8,a7cda650641a4271fba8f12bceef3cbf,...,9f92499d14c6fdba6881e7dc9e6edb97,6c0aed0f589955fe8a49de77e935f62c,d04cf36ea1950c41c1477f00bd29e0a2,0cf8c7a691d536b732042ff3db2f1023,2e705095056dec78df05515f62656880,4e68b29b49a21c8f55390154db8e2756,7a91a16c8662352508c0f2cfde2f6f42,b3d4859e6d92a76c007fc8e7d9b2aa92,355e2bf3c1d973f919f3a979d966b25c,7d77e5454ea232d0be01868a0b4be267
0,**Family Guy** vs **The Simpsons**\r\n\r\n*Rat...,0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,**Cement** vs **Concrete**\r\n\r\n*Rate the se...,1,2.0,NaN,NaN,3.0,2.0,3.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,**chocolate** vs **tea**\r\n\r\n*Rate the sent...,2,2.0,3.0,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,**BMW** vs **Audi**\r\n\r\n*Rate the sentence ...,3,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,**kids** vs **adults**\r\n\r\n*Rate the senten...,4,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1724,**concrete** vs **wood**\r\n\r\n*Rate the sent...,1724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1725,**perl** vs **python**\r\n\r\n*Rate the senten...,1725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1726,**skiing** vs **snowboarding**\r\n\r\n*Rate th...,1726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1727,**baseball** vs **softball**\r\n\r\n*Rate the ...,1727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Count Fleiss' kappa

In [33]:
sentence = df_raters.loc[:, df_raters.columns != 'text']

### Here we select the annotators

In [34]:
annotators = list(df_raters.columns)[2:]
sentence = sentence[annotators]

In [36]:
sentence = sentence[annotators]
all_data = sentence[~sentence.isnull().any(axis=1)]
# print(all_data)
if len(all_data) > 0:
    print(fleiss_kappa(aggregate_raters(all_data)[0], method='fleiss'))

## Count Krippendorf's Alpha

In [37]:
sentence.T

,0,1,2,3,4,5,6,7,8,9,...,1719,1720,1721,1722,1723,1724,1725,1726,1727,1728
d7c8ba18b024aadad3c03dc12bceface,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31e4bb0217a21ef6451faaee8847d7c1,2.0,NaN,3.0,3.0,2.0,NaN,NaN,3.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b0e385e880cb3f5033308f1a9d63e5c4,2.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9a8928d0d44faa52cf9e7118122d68a3,NaN,3.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
646ef569491540d55e1f3140841a1a15,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4e68b29b49a21c8f55390154db8e2756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7a91a16c8662352508c0f2cfde2f6f42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b3d4859e6d92a76c007fc8e7d9b2aa92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
355e2bf3c1d973f919f3a979d966b25c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
kd.alpha(sentence.T, level_of_measurement='interval')

0.3350446915451717

In [39]:
sentence

,d7c8ba18b024aadad3c03dc12bceface,31e4bb0217a21ef6451faaee8847d7c1,b0e385e880cb3f5033308f1a9d63e5c4,9a8928d0d44faa52cf9e7118122d68a3,646ef569491540d55e1f3140841a1a15,2273fc122c3fd60f9e096cf0afaeabb9,5f1770836609d188639b9b73f25500d8,a7cda650641a4271fba8f12bceef3cbf,ad486b05c2371539edf2f43c139db430,437cb936254a18312f9c5cb2ef17310b,...,9f92499d14c6fdba6881e7dc9e6edb97,6c0aed0f589955fe8a49de77e935f62c,d04cf36ea1950c41c1477f00bd29e0a2,0cf8c7a691d536b732042ff3db2f1023,2e705095056dec78df05515f62656880,4e68b29b49a21c8f55390154db8e2756,7a91a16c8662352508c0f2cfde2f6f42,b3d4859e6d92a76c007fc8e7d9b2aa92,355e2bf3c1d973f919f3a979d966b25c,7d77e5454ea232d0be01868a0b4be267
0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,NaN,NaN,3.0,2.0,3.0,3.0,3.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Count Cohen's kappa

In [40]:
from sklearn.metrics import cohen_kappa_score
import itertools

In [ ]:
cohen = []

for a1, a2 in itertools.combinations(annotators, 2):
    sentence = df_raters[[a1, a2]]
    all_data = sentence[~sentence.isnull().any(axis=1)]
    list1 = [tags[i] for i in all_data[a1].values.tolist()]
    list2 = [tags[i] for i in all_data[a2].values.tolist()]
    if str(cohen_kappa_score(list1, list2, labels=[0,1,2,3,4])) != 'nan':
        cohen.append(cohen_kappa_score(list1, list2, labels=[0,1,2,3,4]))

In [42]:
np.mean(cohen), np.median(cohen)

(0.16959827587164003, 0.0)

## ChatGPT agreement

In [65]:
#! /usr/bin/env python
# -*- coding: utf-8
'''
Python implementation of Krippendorff's alpha -- inter-rater reliability

(c)2011-17 Thomas Grill (http://grrrr.org)

Python version >= 2.4 required
'''

from __future__ import print_function
try:
    import numpy as np
except ImportError:
    np = None


def nominal_metric(a, b):
    return a != b


def interval_metric(a, b):
    return (a-b)**2


def ratio_metric(a, b):
    return ((a-b)/(a+b))**2


def krippendorff_alpha(data, metric=interval_metric, force_vecmath=False, convert_items=float, missing_items=None):
    '''
    Calculate Krippendorff's alpha (inter-rater reliability):

    data is in the format
    [
        {unit1:value, unit2:value, ...},  # coder 1
        {unit1:value, unit3:value, ...},   # coder 2
        ...                            # more coders
    ]
    or
    it is a sequence of (masked) sequences (list, numpy.array, numpy.ma.array, e.g.) with rows corresponding to coders and columns to items

    metric: function calculating the pairwise distance
    force_vecmath: force vector math for custom metrics (numpy required)
    convert_items: function for the type conversion of items (default: float)
    missing_items: indicator for missing items (default: None)
    '''

    # number of coders
    m = len(data)

    # set of constants identifying missing values
    if missing_items is None:
        maskitems = []
    else:
        maskitems = list(missing_items)
    if np is not None:
        maskitems.append(np.ma.masked_singleton)

    # convert input data to a dict of items
    units = {}
    for d in data:
        try:
            # try if d behaves as a dict
            diter = d.items()
        except AttributeError:
            # sequence assumed for d
            diter = enumerate(d)

        for it, g in diter:
            if g not in maskitems:
                try:
                    its = units[it]
                except KeyError:
                    its = []
                    units[it] = its
                its.append(convert_items(g))


    units = dict((it, d) for it, d in units.items() if len(d) > 1)  # units with pairable values
    n = sum(len(pv) for pv in units.values())  # number of pairable values

    if n == 0:
        raise ValueError("No items to compare.")

    np_metric = (np is not None) and ((metric in (interval_metric, nominal_metric, ratio_metric)) or force_vecmath)

    Do = 0.
    for grades in units.values():
        if np_metric:
            gr = np.asarray(grades)
            Du = sum(np.sum(metric(gr, gri)) for gri in gr)
        else:
            Du = sum(metric(gi, gj) for gi in grades for gj in grades)
        Do += Du/float(len(grades)-1)
    Do /= float(n)

    if Do == 0:
        return 1.

    De = 0.
    for g1 in units.values():
        if np_metric:
            d1 = np.asarray(g1)
            for g2 in units.values():
                De += sum(np.sum(metric(d1, gj)) for gj in g2)
        else:
            for g2 in units.values():
                De += sum(metric(gi, gj) for gi in g1 for gj in g2)
    De /= float(n*(n-1))

    return 1.-Do/De if (Do and De) else 1.

In [71]:
# --- Fleiss' Kappa Preparation ---
def to_fleiss_format(human, gpt):
    ratings = list(zip(human, gpt))
    categories = sorted(set(human + gpt))
    category_index = {cat: idx for idx, cat in enumerate(categories)}
    
    fleiss_data = []
    for h, g in ratings:
        row = [0] * len(categories)
        row[category_index[h]] += 1
        row[category_index[g]] += 1
        fleiss_data.append(row)
    return fleiss_data


In [74]:
import pandas as pd
import numpy as np
from collections import Counter
from itertools import combinations, product
from statsmodels.stats.inter_rater import fleiss_kappa

# Load Excel sheet "контрольные"
file_path = "data/expert_annotation.xlsx"  # <- Adjust this path if needed
control_df = pd.read_excel(file_path, sheet_name="контрольные")

# Select annotator columns (you can rename these if needed)
annotator_columns = ['relevance', 'relevance.1', 'relevance.2', 'relevance.3']
ratings_df = control_df[annotator_columns].dropna().astype(int)


fleiss_data = to_fleiss_format_from_matrix(ratings_df)
fleiss_result = fleiss_kappa(fleiss_data)


# --- Output ---
print(f"Fleiss' Kappa: {fleiss_result:.4f}")


Fleiss' Kappa: 0.8172


In [109]:
import pandas as pd
import json

# Load control data
control_arguments = control_df["arguments"].dropna().tolist()

# Load GPT-4 dataset
with open("data/dataset_cqa_all_gpt.json", "r") as f:
    gpt_data = json.load(f)

# Flatten GPT data: get list of (argument_text, relevance_gpt)
gpt4_entries = []
for entry in gpt_data:
    for arg in entry["arguments"]:
        if arg["argument"] is not None and arg["relevance_gpt"] is not None:
            gpt4_entries.append((arg["argument"].lower(), int(arg["relevance_gpt"])))

# Create a lookup from argument to GPT-4 score
gpt4_lookup = {text: score for text, score in gpt4_entries}

# Match control arguments with GPT-4 annotations
trimmed_gpt4 = []
for arg in control_arguments:
    cleaned = str(arg).lower()
    trimmed_gpt4.append(gpt4_lookup.get(cleaned, None))  # None if not found


In [110]:
# Columns containing human annotations
control_df['gpt4'] = trimmed_gpt4


In [111]:
ratings_df = control_df[annotator_columns+['gpt4']].dropna().astype(int)


fleiss_data = to_fleiss_format_from_matrix(ratings_df)
fleiss_result = fleiss_kappa(fleiss_data)

fleiss_result

0.627659574468085

In [112]:
ratings_df = control_df[['(supposed) relevance','gpt4']].dropna().astype(int)

fleiss_data = to_fleiss_format_from_matrix(ratings_df)
fleiss_result = fleiss_kappa(fleiss_data)

fleiss_result

0.4636015325670499